In [1]:
import os, sys, json, argparse
import pandas as p
import numpy as np
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
from string import punctuation

import urllib.request, html5lib
from bs4 import BeautifulSoup, Tag, UnicodeDammit

In [57]:
#first i found a routine that extracted a json version of a script and converted it something usable
#used the same author to extract the scripts from http://www.imsdb.com/scripts into a json file
#the parsing routine started out as an interactive py and i converted it to a batch process. 

SW5 = '/Users/kenmckee/Desktop/SWE5'
SW7 = '/Users/kenmckee/Desktop/SWE7ANA'

with open('/Users/kenmckee/Desktop/TFA') as fd:
    script = json.load(fd)
    
    print(script[])

TypeError: unhashable type: 'slice'

In [21]:
BLOCK_TYPES=['character', 'speech', 'stage direction', 'location']
CHARACTER=0
SPEECH=1
DIRECTIONS=2
LOCATION=3

In [22]:
#adapted this routine from https://github.com/Adrien-Luxey/Da-Fonky-Movie-Script-Parser

def extract_characters(script):
    '''
    Extracts the (unique) characters list from the script
    '''
    characters=[]
    for block in script['movie_script']:
        if(block['type'] == BLOCK_TYPES[SPEECH]):
            character = block['character']
            if not character in characters:
                characters.append(character)

    return characters

def extract_locations(script):
    '''
    Extracts the (unique) locations list from the script
    '''
    locations=[]
    for block in script['movie_script']:
        if(block['type'] == BLOCK_TYPES[LOCATION]):
            location = block['text']
            if not location in locations:
                locations.append(location)

    return locations

def extract_directions(script):
    '''
    Extracts the stage directions list from the script
    '''
    directions=[]
    for block in script['movie_script']:
        if(block['type'] == BLOCK_TYPES[DIRECTIONS]):
            directions.append(block['text'])

    return directions

def extract_speech_given_character(script, character, strict_match=False):
    '''
    Extracts the given character's utterances from the script

    If strict_match is True, we will only extract utterances that perfectly match (==) the parameter;
    otherwise, we will extract utterances whose character partly matches (in) the parameter.
    In both cases, the match is case-insensitive.

    Also asks the user wether one wants to keep the character's name before each utterance.
    '''

    speeches=[]
    for block in script['movie_script']:
        if( block['type'] == BLOCK_TYPES[SPEECH] and
            (strict_match and (character.lower() == block['character'].lower()) or
             not strict_match and (character.lower() in block['character'].lower())) ):
            if( keep_character_name ):
                speeches.append(block['character'])
            speeches.append(block['text'])

    return speeches

def extract_all_characters_speech(script):
    '''
    Extracts all speeches from the script
    '''
    return extract_speech_given_character(script, '')

def extract_speech_asking_user(script):
    '''
    Extracts utterances by asking the user which character one wants to get

    Also asks wether the user wants a perfect (==) or partial (in) match.
    '''
    character = input_string('Please provide the name of the character: ')

    strict_match = False
    answer = input('Do you want utterances of this exact character (or any character that matches \'{}\')? (y/N) '.format(character))
    if( answer == 'y' or answer =='Y' ):
        strict_match = True

    return extract_speech_given_character(script, character, strict_match)

def extract_speech_using_characters_list(script):
    '''
    Extracts utterances by providing the user with the characters list

    Also asks wether the user wants a perfect (==) or partial (in) match.
    '''
    characters = sorted(extract_characters(script))
    character = characters[input_from_list('Please choose a character:', characters)]

    strict_match = False
    answer = input('Do you want utterances of this exact character (or any character that matches \'{}\')? (y/N) '.format(character))
    if( answer == 'y' or answer =='Y' ):
        strict_match = True

    return extract_speech_given_character(script, character, strict_match)

def extract_speech(script):
    '''
    Asks the user which speeches one wants to extract, and calls the appropriate function
    '''
    speech=[]

    choices = ['all characters',
               'give the character\'s name',
               'choose from the characters list']

    action = input_from_list("Which character speeches do you want to extract?", choices)

    if( action == 0 ):
        return extract_all_characters_speech(script)
    elif( action == 1 ):
        return extract_speech_asking_user(script)
    elif( action == 2 ):
        return extract_speech_using_characters_list(script)





In [149]:
keep_character_name = True
strict_match = False

result_character = extract_characters(script)
result_direction = extract_directions(script)
result_location = extract_locations(script)
result_speech = extract_all_characters_speech(script)
print(result_character)

#print(result_location)
#print(result_location)




['LOR SAN TEKKA', 'POE', "POE (CONT'D)", 'HIM:\nSENIOR TROOPER', 'KYLO REN', '(CONTINUED)', "POE\n(INDICATES KYLO'S\nMASK)", 'KYLO REN\n(TO TROOPERS)', 'STORMTROOPER #1', 'CAPTAIN PHASMA', 'STORMTROOPER #2', 'FN-2187', 'UNKAR', 'REY', "REY (CONT'D)", 'STRAIGHTENS THE\nANTENNA)', '(BB-8 BEEPS)', '(HE BEEPS)', 'GENERAL HUX', 'REY\n(OBJECTS)', "UNKAR (CONT'D)", '(TO BB-8)', 'STORMTROOPER', 'POE\n(COMING TO)', "(THAT'S CRAZY)", '(BEAT)', 'FINN', 'LIEUTENANT MITAKA', '(VEXED)', 'TECHNICIAN #1', 'TECHNICIAN #1\n(GETTING READINGS)', "FINN (CONT'D)", 'GENERAL HUX\n(WITH VITRIOL)', 'GENERAL HUX\n(YOU SON-OF-A-BITCH)', 'AGAIN)', '(RELIVING IT)', 'ROLLS OFF)', '(WHISPERS)', 'REY\n(TO BB-8)', 'OFF-CAMERA)', 'FINN\n(CLIMBS DOWN)', '(TO HERSELF)', '(TO FINN)', '(QUIETLY)', 'REY\n(HEADS OFF)', 'HAN', "HAN (CONT'D)", 'HAN\n(TO CHEWIE)', '(TO REY)', 'FINN\n(TO CHEWIE)', 'THE HYPERDRIVE--\nHAN\n(OVERLAPPING HER)', '(CHEWIE MOAN-TALKS)', 'BALA-TIK', 'TASU LEECH', 'RAZOO QIN-FEE', 'REY\n(PALE)', 'REY\n(SH

In [25]:
keep_character_name = True
strict_match = False

text=[]
characters=[]
for character in result_character:
    for block in script['movie_script']:
        if( block['type'] == BLOCK_TYPES[SPEECH] and
            (strict_match and (character.lower() == block['character'].lower()) or
             not strict_match and (character.lower() in block['character'].lower())) ):
            if( keep_character_name ):
                characters.append(block['character'])
                text.append(block['text'])

In [26]:
textl = []
for line in text:
        line2 = line.replace('\n',' ') 
        textl.append(line2)
        
print(textl)

["This will begin to make things right. I've traveled too far, and seen too much, to ignore the despair in the galaxy. Without the Jedi, there can be no balance in the Force.", '"The General." To me, she\'s royalty. Well, she certainly is that.', 'You have to leave. Go!', 'Something far worse has happened to you.', 'The First Order rose from the dark side... you did not.', 'You may try, but you cannot deny the truth that is your family.', "Well, because of you now we have a chance. The General's been after this for a long time.", "We've got company.", 'You have to hide.', 'Come on, BB-8! Hurry!', "I see 'em!", "You take this. It's safer with you than it is with me. You get as far away from here as you can. Do you hear me? (BB-8 BEEPS, hesitates) I'll come back for you! It will be alright.", 'So who talks first? You talk first?', "It's just very hard to understand you with all the...", '... Apparatus.', 'All right. All right!', 'Not really.', 'Might wanna rethink your technique.', 'The 

In [27]:
type(textl)
textll = " ".join(textl)
type(textll)

str

In [28]:
#print(allwords_tokenized)

print(textll)

This will begin to make things right. I've traveled too far, and seen too much, to ignore the despair in the galaxy. Without the Jedi, there can be no balance in the Force. "The General." To me, she's royalty. Well, she certainly is that. You have to leave. Go! Something far worse has happened to you. The First Order rose from the dark side... you did not. You may try, but you cannot deny the truth that is your family. Well, because of you now we have a chance. The General's been after this for a long time. We've got company. You have to hide. Come on, BB-8! Hurry! I see 'em! You take this. It's safer with you than it is with me. You get as far away from here as you can. Do you hear me? (BB-8 BEEPS, hesitates) I'll come back for you! It will be alright. So who talks first? You talk first? It's just very hard to understand you with all the... ... Apparatus. All right. All right! Not really. Might wanna rethink your technique. The Resistance will not be intimidated by you. If -- what--? 

In [29]:
def tokenize():
    if textll is not None:
        words = textll.lower().split()
        return words
    else:
        return None

def map_book(tokens):
    hash_map = {}

    if tokens is not None:
        for element in tokens:
            # Remove Punctuation
            word = element.replace(",","")
            word = word.replace(".","")

            # Word Exist?
            if word in hash_map:
                hash_map[word] = hash_map[word] + 1
            else:
                hash_map[word] = 1

        return hash_map
    else:
        return None

In [30]:
import operator
# Tokenize the Book
textlll = tokenize()


# Create a Hash Map (Dictionary)
map = map_book(textlll)

smap = sorted(map.items(), key=operator.itemgetter(1), reverse=True)

print(smap)

[('the', 353), ('you', 287), ('to', 190), ('i', 190), ('a', 120), ('it', 118), ('we', 99), ('that', 92), ('is', 83), ('this', 81), ('no', 80), ('and', 76), ('of', 75), ('--', 74), ('in', 63), ('on', 61), ('me', 60), ('what', 58), ('get', 57), ("i'm", 56), ('was', 56), ('for', 54), ('with', 54), ('have', 51), ('your', 49), ('are', 49), ('can', 45), ("you're", 43), ('not', 42), ('him', 41), ('do', 40), ('be', 39), ('back', 39), ('first', 38), ('my', 38), ('know', 37), ('will', 35), ('come', 35), ('he', 35), ('from', 34), ('see', 34), ('all', 34), ('but', 33), ('go', 33), ("don't", 33), ("it's", 31), ('need', 31), ('got', 30), ('one', 30), ('there', 29), ('droid', 28), ('at', 28), ('our', 28), ('if', 25), ('order', 24), ('here', 24), ('so', 24), ('resistance', 24), ('as', 23), ('just', 23), ("we're", 23), ('how', 23), ("i've", 22), ('too', 21), ('us', 21), ('finn', 21), ('good', 21), ("that's", 21), ('han', 21), ('solo', 21), ('rey', 21), ('been', 20), ('take', 20), ('bb-8', 20), ('did', 

In [36]:
#token count
len(textlll)
len(textlll)


8215

In [32]:
word_list = ['the','you','too']
# Show Word Information
for word in word_list:
    print('Word: [' + word + '] Frequency: ' + str(map[word]))

#print(tokens)

Word: [the] Frequency: 353
Word: [you] Frequency: 287
Word: [too] Frequency: 21


In [35]:
#Term Frequency Analysis

stoplistx = []
import nltk
from nltk.probability import FreqDist
ndist = FreqDist(textlll)
nitems = ndist.most_common(70)
for item in nitems:
    print (item[0], '\t', item[1])
    stoplistx.append(item[0])
print(stoplistx)
type(stoplistx)

the 	 351
you 	 240
to 	 190
i 	 189
a 	 120
we 	 99
that 	 81
is 	 77
and 	 76
it 	 76
of 	 75
-- 	 74
this 	 72
in 	 63
what 	 58
get 	 57
i'm 	 55
with 	 54
was 	 53
for 	 52
on 	 51
have 	 50
your 	 49
are 	 49
you're 	 43
can 	 40
not 	 39
be 	 38
first 	 38
it. 	 38
no 	 37
my 	 37
you. 	 36
he 	 35
come 	 34
will 	 33
do 	 33
from 	 32
don't 	 32
see 	 31
it's 	 31
know 	 31
all 	 30
need 	 30
but 	 29
got 	 28
our 	 28
me 	 28
back 	 27
one 	 27
at 	 27
if 	 25
go 	 25
me. 	 24
just 	 23
we're 	 23
no, 	 23
how 	 23
i've 	 22
as 	 22
so 	 22
that's 	 21
him. 	 21
been 	 20
take 	 20
droid 	 19
they 	 19
... 	 18
him 	 18
(cont'd) 	 18
['the', 'you', 'to', 'i', 'a', 'we', 'that', 'is', 'and', 'it', 'of', '--', 'this', 'in', 'what', 'get', "i'm", 'with', 'was', 'for', 'on', 'have', 'your', 'are', "you're", 'can', 'not', 'be', 'first', 'it.', 'no', 'my', 'you.', 'he', 'come', 'will', 'do', 'from', "don't", 'see', "it's", 'know', 'all', 'need', 'but', 'got', 'our', 'me', 'back', 'o

list

In [77]:
#Conditional Probabilities


from collections import defaultdict

def build_conditional_probabilities(corpus):
	"""
	The function takes as its input a corpus string (words separated by 
	spaces) and returns a 2D dictionnary of probabilities P(next|current) of
	seeing a word "next" conditionnaly to seeing a word "current". 
	"""

	# First we parse the string to build a double dimension dictionnary that
	# returns the conditional probabilities.

	# We parse the string to build a first dictionnary indicating for each
	# word, what are the words that follow it in the string. Repeated next
	# words are kept so we use a list and not a set. 

	tokenized_string = corpus.split()
	previous_word = ""
	dictionnary = defaultdict(list)

	for current_word in tokenized_string:
		if previous_word != "":
			dictionnary[previous_word].append(current_word)
		previous_word = current_word
		
	# We know parse dictionnary to compute the probability each observed
	# next word for each word in the dictionnary. 

	for key in dictionnary.keys():
		next_words = dictionnary[key]
		unique_words = set(next_words) # removes duplicated
		nb_words = len(next_words)
		probabilities_given_key = {}
		for unique_word in unique_words:
			probabilities_given_key[unique_word] = \
				float(next_words.count(unique_word)) / nb_words
		dictionnary[key] = probabilities_given_key

	return dictionnary


def bigram_next_word_predictor(conditional_probabilities, current, next_candidate):
	"""
	The function takes as its input a 2D dictionnary of probabilities 
	P(next|current) of seeing a word "next" conditionnaly to seeing a word 
	"current", the current word being read, and a next candidate word, and
	returns P(next_candidate|current).
	"""

	# We look for the probability corresponding to the 
	# current -> next_candidate pair

	if current in conditional_probabilities:
		if next_candidate in conditional_probabilities[current]:
			return conditional_probabilities[current][next_candidate]

	#If current -> next_candidate pair has not been observed in the corpus,
	#the corresponding dictionnary keys will not be defined. We return 
	#a probability 0.0

	return 0.0

# An example corpus to try out the function
# We call the conditional probability dictionnary builder function
conditional_probabilities = build_conditional_probabilities(textll)
#type(conditional_probabilities)
#print(conditional_probabilities)

print(conditional_probabilities['Force'])
print(conditional_probabilities['Force!'])
print(conditional_probabilities['Force,'])
print(conditional_probabilities['Force.'])

{'be': 0.6666666666666666, 'works--!': 0.3333333333333333}
{'Come': 0.5, 'I': 0.5}
{'untrained': 0.5, 'the': 0.5}
{'"The': 0.3333333333333333, 'Finn!': 0.3333333333333333, 'It': 0.3333333333333333}


In [61]:
bigramx = bigram_next_word_predictor(conditional_probabilities, "the", "Force")
bigramy = bigram_next_word_predictor(conditional_probabilities, "the", "force")



#print(conditional_probabilities)
print(bigramx)
print(bigramy)

In [58]:
# this function takes a string and a list of words as parameters.
#   It will return all the words in the list that contain the string as a substring
def searchstring (substring, wordlist):
    # initialize the result
    result = [ ]
    #  loop over all the words
    for word in wordlist:
        # test each word if it contains the substring
        if substring in word:
            # add it to the result
            result.append(word)
    return result


In [75]:
result = searchstring('Force', textl)

for string in result:
    print(string)

    #print(result)

This will begin to make things right. I've traveled too far, and seen too much, to ignore the despair in the galaxy. Without the Jedi, there can be no balance in the Force.
She's strong with the Force, untrained but, stronger than she knows.
You need a teacher! I can show you the ways of the Force!
The Force.
'Crazy thing is, it's true. The Force, the Jedi, all of it. It's all true.
Solo, we'll figure it out! We'll use the Force!
That's not how the Force works--!
The belonging you seek is not behind you. It is ahead. I am no Jedi, but I know the Force. It moves through and surrounds every living thing. Close your eyes. Feel it.
May the Force be with you.
May the Force be with you.


In [39]:
# function that takes a word and returns true if it consists only
#   of non-alphabetic characters

def alpha_filter(w):
  # pattern to match a word of non-alphabetical characters
    pattern = re.compile('^[^a-z]+$')
    if (pattern.match(w)):
        return True
    else:
        return False


In [40]:
alphawords = [w for w in textlll if not alpha_filter(w)]
print(len(alphawords))
print(alphawords[:100])


8113
['this', 'will', 'begin', 'to', 'make', 'things', 'right.', "i've", 'traveled', 'too', 'far,', 'and', 'seen', 'too', 'much,', 'to', 'ignore', 'the', 'despair', 'in', 'the', 'galaxy.', 'without', 'the', 'jedi,', 'there', 'can', 'be', 'no', 'balance', 'in', 'the', 'force.', '"the', 'general."', 'to', 'me,', "she's", 'royalty.', 'well,', 'she', 'certainly', 'is', 'that.', 'you', 'have', 'to', 'leave.', 'go!', 'something', 'far', 'worse', 'has', 'happened', 'to', 'you.', 'the', 'first', 'order', 'rose', 'from', 'the', 'dark', 'side...', 'you', 'did', 'not.', 'you', 'may', 'try,', 'but', 'you', 'cannot', 'deny', 'the', 'truth', 'that', 'is', 'your', 'family.', 'well,', 'because', 'of', 'you', 'now', 'we', 'have', 'a', 'chance.', 'the', "general's", 'been', 'after', 'this', 'for', 'a', 'long', 'time.', "we've", 'got']


In [41]:
nltkstopwords = nltk.corpus.stopwords.words('english')
print(len(nltkstopwords))
print(nltkstopwords)


179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

In [42]:
print(len(textlll))
print("")
print(textlll[:100])
print("")
print(textlll[12148:12158])


8215

['this', 'will', 'begin', 'to', 'make', 'things', 'right.', "i've", 'traveled', 'too', 'far,', 'and', 'seen', 'too', 'much,', 'to', 'ignore', 'the', 'despair', 'in', 'the', 'galaxy.', 'without', 'the', 'jedi,', 'there', 'can', 'be', 'no', 'balance', 'in', 'the', 'force.', '"the', 'general."', 'to', 'me,', "she's", 'royalty.', 'well,', 'she', 'certainly', 'is', 'that.', 'you', 'have', 'to', 'leave.', 'go!', 'something', 'far', 'worse', 'has', 'happened', 'to', 'you.', 'the', 'first', 'order', 'rose', 'from', 'the', 'dark', 'side...', 'you', 'did', 'not.', 'you', 'may', 'try,', 'but', 'you', 'cannot', 'deny', 'the', 'truth', 'that', 'is', 'your', 'family.', 'well,', 'because', 'of', 'you', 'now', 'we', 'have', 'a', 'chance.', 'the', "general's", 'been', 'after', 'this', 'for', 'a', 'long', 'time.', "we've", 'got']

[]


In [43]:
morestopwords = stoplistx


In [44]:
stopwords =  morestopwords
print(len(stopwords))
print(stopwords)


70
['the', 'you', 'to', 'i', 'a', 'we', 'that', 'is', 'and', 'it', 'of', '--', 'this', 'in', 'what', 'get', "i'm", 'with', 'was', 'for', 'on', 'have', 'your', 'are', "you're", 'can', 'not', 'be', 'first', 'it.', 'no', 'my', 'you.', 'he', 'come', 'will', 'do', 'from', "don't", 'see', "it's", 'know', 'all', 'need', 'but', 'got', 'our', 'me', 'back', 'one', 'at', 'if', 'go', 'me.', 'just', "we're", 'no,', 'how', "i've", 'as', 'so', "that's", 'him.', 'been', 'take', 'droid', 'they', '...', 'him', "(cont'd)"]


In [143]:
textllll = [w for w in textlll if not w in stopwords]
print(len(stoppwords))


4606


In [133]:
dist = FreqDist(stoppwords)
items = dist.most_common(30)
for item in items:
  print(item)



('about', 18)
('han', 18)
('like', 18)
('who', 17)
('it!', 17)
('going', 17)
('no.', 17)
('never', 17)
('there', 16)
('where', 16)
('did', 15)
('gonna', 15)
('out', 15)
('their', 15)
("there's", 15)
('his', 15)
("can't", 15)
('too', 14)
('has', 14)
('order', 14)
('us', 14)
("we'll", 14)
('yeah,', 14)
('tell', 14)
('oh', 14)
("he's", 13)
("i'll", 12)
('it?', 12)
('good', 12)
('luke', 12)


In [144]:
bigramsx = list(nltk.bigrams(textllll))

#print(bigramsx[:20])

bbgram = []
fdist = nltk.FreqDist(bigramsx)
for k,v in fdist.items():
    if float(v) > 4:
        bbgram.append(k)
        bbgram.append(v)
print(bbgram)

dicts = {}
keys = range(4)
values = ["Hi", "I", "am", "John"]
for i in keys:
    for x in values:
        dicts[i] = x
print(dicts)

[('general', 'hux'), 5, ('death', 'star.'), 5, ('kylo', 'ren'), 9, ('han', 'solo'), 5]


In [98]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()



In [96]:
finder = BigramCollocationFinder.from_words(textlll)
scored = finder.score_ngrams(bigram_measures.raw_freq)


In [122]:
#print(scored)
type(scored)

scored.sort()

print(sscored)
#sscored =scored.sort(key=lambda tup: tup[2], reverse=True)
#print(sscored)

None


In [130]:
def getKey(item):
    return item[1]
#sorted(scored, key=getKey, reverse = True)
sorted(scored, key=getKey, reverse = True)

[(('the', 'first'), 0.003530127814972611),
 (('are', 'you'), 0.003286670724284845),
 (('in', 'the'), 0.002434570906877663),
 (('with', 'the'), 0.002434570906877663),
 (('to', 'get'), 0.0021911138161898967),
 (('to', 'the'), 0.0021911138161898967),
 (('of', 'the'), 0.0019476567255021302),
 (('i', 'can'), 0.0018259281801582471),
 (('back', 'to'), 0.0015824710894704807),
 (('first', 'order'), 0.0015824710894704807),
 (('have', 'to'), 0.0014607425441265976),
 (('this', 'is'), 0.0014607425441265976),
 (('i', 'got'), 0.0013390139987827145),
 (('i', 'need'), 0.0013390139987827145),
 (('on', 'the'), 0.0013390139987827145),
 (('we', 'have'), 0.0013390139987827145),
 (('you', 'see'), 0.0013390139987827145),
 (('it', 'is'), 0.0012172854534388314),
 (('the', 'droid'), 0.0012172854534388314),
 (('we', 'can'), 0.0012172854534388314),
 (('what', 'are'), 0.0012172854534388314),
 (('do', 'you'), 0.0010955569080949483),
 (('from', 'the'), 0.0010955569080949483),
 (('going', 'to'), 0.0010955569080949483)

In [131]:
finder.apply_word_filter(alpha_filter)
scored = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:50]:
    print (bscore)


(('the', 'first'), 0.003530127814972611)
(('are', 'you'), 0.003286670724284845)
(('in', 'the'), 0.002434570906877663)
(('with', 'the'), 0.002434570906877663)
(('to', 'get'), 0.0021911138161898967)
(('to', 'the'), 0.0021911138161898967)
(('of', 'the'), 0.0019476567255021302)
(('i', 'can'), 0.0018259281801582471)
(('back', 'to'), 0.0015824710894704807)
(('first', 'order'), 0.0015824710894704807)
(('have', 'to'), 0.0014607425441265976)
(('this', 'is'), 0.0014607425441265976)
(('i', 'got'), 0.0013390139987827145)
(('i', 'need'), 0.0013390139987827145)
(('on', 'the'), 0.0013390139987827145)
(('we', 'have'), 0.0013390139987827145)
(('you', 'see'), 0.0013390139987827145)
(('it', 'is'), 0.0012172854534388314)
(('the', 'droid'), 0.0012172854534388314)
(('we', 'can'), 0.0012172854534388314)
(('what', 'are'), 0.0012172854534388314)
(('do', 'you'), 0.0010955569080949483)
(('from', 'the'), 0.0010955569080949483)
(('going', 'to'), 0.0010955569080949483)
(('got', 'a'), 0.0010955569080949483)
(('i', '

In [145]:
finder.apply_word_filter(lambda w: w in stopwords)
scored = finder.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:50]:
    print (bscore)


(('kylo', 'ren'), 0.0010955569080949483)
(('death', 'star.'), 0.0006086427267194157)
(('general', 'hux'), 0.0006086427267194157)
(('han', 'solo'), 0.0006086427267194157)
(('dark', 'side.'), 0.00048691418137553256)
(('door', 'open.'), 0.00048691418137553256)
(('han', 'solo.'), 0.00048691418137553256)
(('remove', 'these'), 0.00048691418137553256)
(('someone', 'who'), 0.00048691418137553256)
(('tell', 'us'), 0.00048691418137553256)
(('these', 'restraints.'), 0.00048691418137553256)
(('too', 'long.'), 0.00048691418137553256)
(('(he', 'beeps'), 0.0003651856360316494)
(('away', 'too'), 0.0003651856360316494)
(('back.', 'but...'), 0.0003651856360316494)
(('bb-8', 'says'), 0.0003651856360316494)
(('beeps', 'again)'), 0.0003651856360316494)
(('but...', "there's"), 0.0003651856360316494)
(('c-3po', '(o.s.)'), 0.0003651856360316494)
(('chewie,', 'check'), 0.0003651856360316494)
(('coming', 'back.'), 0.0003651856360316494)
(('dear', 'child.'), 0.0003651856360316494)
(("he's", 'carrying'), 0.000365

In [146]:
finder2 = BigramCollocationFinder.from_words(textllll)
finder2.apply_freq_filter(5)
scored = finder2.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:50]:
    print (bscore)




(('kylo', 'ren'), 0.0019539730785931393)
(('death', 'star.'), 0.0010855405992184109)
(('general', 'hux'), 0.0010855405992184109)
(('han', 'solo'), 0.0010855405992184109)


In [140]:
finder2.apply_ngram_filter(lambda w1, w2: len(w1) > 5)
scored = finder2.score_ngrams(bigram_measures.raw_freq)
for bscore in scored[:50]:
    print (bscore)

(('the', 'first'), 0.003530127814972611)
(('are', 'you'), 0.003286670724284845)
(('in', 'the'), 0.002434570906877663)
(('with', 'the'), 0.002434570906877663)
(('to', 'get'), 0.0021911138161898967)
(('to', 'the'), 0.0021911138161898967)
(('of', 'the'), 0.0019476567255021302)
(('i', 'can'), 0.0018259281801582471)
(('back', 'to'), 0.0015824710894704807)
(('first', 'order'), 0.0015824710894704807)
(('have', 'to'), 0.0014607425441265976)
(('this', 'is'), 0.0014607425441265976)
(('i', 'got'), 0.0013390139987827145)
(('i', 'need'), 0.0013390139987827145)
(('on', 'the'), 0.0013390139987827145)
(('we', 'have'), 0.0013390139987827145)
(('you', 'see'), 0.0013390139987827145)
(('it', 'is'), 0.0012172854534388314)
(('the', 'droid'), 0.0012172854534388314)
(('we', 'can'), 0.0012172854534388314)
(('what', 'are'), 0.0012172854534388314)
(('do', 'you'), 0.0010955569080949483)
(('from', 'the'), 0.0010955569080949483)
(('going', 'to'), 0.0010955569080949483)
(('got', 'a'), 0.0010955569080949483)
(('i', '

In [147]:
finder3 = BigramCollocationFinder.from_words(textllll)
scored = finder3.score_ngrams(bigram_measures.pmi)
for bscore in scored[:50]:
    print (bscore)

(('"need', 'know"'), 12.169298695792845)
(('"personal', 'interests"'), 12.169298695792845)
(('"that', 'thing"'), 12.169298695792845)
(('"the', 'general."'), 12.169298695792845)
(('(a', 'nervous'), 12.169298695792845)
(('(at', 'hesitant)'), 12.169298695792845)
(('(beat)', 'blast!'), 12.169298695792845)
(('(chewie', 'grabs'), 12.169298695792845)
(('(cocky)', 'perfect!'), 12.169298695792845)
(('(han', 'starts'), 12.169298695792845)
(('(hard', 'say)'), 12.169298695792845)
(('(improvising,', 'fast)'), 12.169298695792845)
(('(it', 'hits'), 12.169298695792845)
(('(laughs)', '(cocky)'), 12.169298695792845)
(('(louder,', 'waving)'), 12.169298695792845)
(('(points', 'horizon)'), 12.169298695792845)
(('(reaches', 'out,'), 12.169298695792845)
(('(reacts', 'seat)'), 12.169298695792845)
(('(rey', 'stares'), 12.169298695792845)
(('(searching', 'tape)'), 12.169298695792845)
(('(shrugs,', 'smiles,'), 12.169298695792845)
(('(stops,', 'fleetingly)'), 12.169298695792845)
(('(throws', 'tool)'), 12.16929869

In [148]:
finder3.apply_freq_filter(5)
scored = finder3.score_ngrams(bigram_measures.pmi)
for bscore in scored[:30]:
    print (bscore)



(('death', 'star.'), 9.361943773735241)
(('general', 'hux'), 9.098909367901449)
(('kylo', 'ren'), 8.420360459960563)
(('han', 'solo'), 7.7363392885167395)


In [ ]:
import csv
dict = script
w = csv.writer(open("SW5.csv", "w"))
for key, val in dict.items():
    w.writerow([key, val])